# seaMass SWATH demo
This notebook illustrates how to load seaMass input/output on SWATH data. Firstly, we need to import the relevant modules...

In [ ]:
using Plots; plotly(size=(980,350))
using Interact

try
    ENV["PYTHON"]=""
    Pkg.checkout("SeaMass")
end
import SeaMass

In [ ]:
reload("SeaMass")

### Compare mzMLb input (sampled) to 'mzmlb2smb' smb input (binned)

In [ ]:
mzmlInputSpectra = SeaMass.MzmlSpectrum[]
maxIntensity = 0.0
maxIntensitySpectrum = 1
for i = 1:100
    push!(mzmlInputSpectra, SeaMass.MzmlSpectrum(
            "../data/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.mzMLb",
            (i - 1) * 51 + 1))    
    maxIntensity_ = maximum(mzmlInputSpectra[i].intensities)   
    if maxIntensity_ > maxIntensity
        maxIntensity = maxIntensity_
        maxIntensitySpectrum = i
    end
end

smbInputSpectra = SeaMass.SmbSpectrum[]
for i = 1:100
    push!(smbInputSpectra, SeaMass.SmbSpectrum(
        "../data/out/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500" * 
            "/1.mzmlb2smb/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.p.smb", i))
end

@manipulate for i = slider(1:100, label="Spectrum", value=maxIntensitySpectrum, continuous_update=true)
    plot(mzmlInputSpectra[i].mzs, mzmlInputSpectra[i].intensities, label="mzML input", m=2)
    plot!(smbInputSpectra[i].locations, vcat(smbInputSpectra[i].counts / smbInputSpectra[i].exposure, 0.0),
        line=:steppost, label="SMB input")
    plot!(xlabel = "m/z (Th)", ylabel = "intensity",
        xlims=(602, 605),
        ylims=(-0.05 * maxIntensity, 1.05 * maxIntensity))
end

### Compare smb input to 'seamass' ⇨ 'seamass-restore' smb output

In [ ]:
smbOutputSpectra = SeaMass.SmbSpectrum[]
smbBinWidths = Array[]
maxCountsDensity = 0.0
maxCountsDensitySpectrum = 1
for i = 1:100
    push!(smbOutputSpectra, SeaMass.SmbSpectrum(
        "../data/out/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500" *
            "/3.seamass-restore/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.p.smb", i))
    push!(smbBinWidths, smbOutputSpectra[i].locations[2:end] - smbOutputSpectra[i].locations[1:end-1])
    
    maxCountsDensity_ = maximum(smbInputSpectra[i].counts ./ smbBinWidths[i])   
    if maxCountsDensity_ > maxCountsDensity
        maxCountsDensity = maxCountsDensity_
        maxCountsDensitySpectrum = i
    end
end

@manipulate for i = slider(1:100, label="Spectrum", value=maxIntensitySpectrum, continuous_update=true)
    plot(smbInputSpectra[i].locations, vcat(smbInputSpectra[i].counts ./ smbBinWidths[i], 0.0),
        line=:steppost, label="SMB input")
    plot!(smbOutputSpectra[i].locations, vcat(smbOutputSpectra[i].counts ./ smbBinWidths[i], 0.0),
        line=:steppost, label="SMB output")
    plot!(xlabel = "m/z (Th)", ylabel = "ion count density",
        xlims=(602, 605),
        ylims=(-0.05 * maxCountsDensity, 1.05 * maxCountsDensity))
end

### Compare smb output to 'seamass-restore' mzMLb output

In [ ]:
mzmlOutputSpectra = SeaMass.MzmlSpectrum[]
maxIntensityDensity = 0.0
maxIntensityDensitySpectrum = 1
for i = 1:100
    push!(mzmlOutputSpectra, SeaMass.MzmlSpectrum(
            "../data/out/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500" * 
                "/4.smb2mzmlb/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.mzMLb", i))
    
    maxIntensityDensity_ = maximum(mzmlOutputSpectra[i].intensities)   
    if maxIntensityDensity_ > maxIntensityDensity
        maxIntensityDensity = maxIntensityDensity_
        maxIntensityDensitySpectrum = i
    end
end

@manipulate for i = slider(1:100, label="Spectrum", value=maxIntensitySpectrum, continuous_update=true)
    plot(smbOutputSpectra[i].locations,
        vcat(smbOutputSpectra[i].counts ./ (smbOutputSpectra[i].exposure * smbBinWidths[i]), 0.0),
        line=:steppost, label="SMB output")
    plot!(mzmlOutputSpectra[i].mzs, mzmlOutputSpectra[i].intensities, label="mzML output", m=2)
    plot!(xlabel = "m/z (Th)", ylabel = "intensity density",
        xlims=(602, 605),
        ylims=(-0.05 * maxIntensityDensity, 1.05 * maxIntensityDensity))
end

### Compare smb input to 'seamass' ⇨ 'seamass-restore --centroid' smb output

In [ ]:
smbCentroidedOutputSpectra = SeaMass.SmbSpectrum[]
for i = 1:100
    push!(smbCentroidedOutputSpectra, SeaMass.SmbSpectrum(
        "../data/out/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500" *
            "/5.seamass-restore_--centroid/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.p.smb", i))
end

@manipulate for i = slider(1:100, label="Spectrum", value=maxIntensitySpectrum, continuous_update=true)
    plot(smbInputSpectra[i].locations, vcat(smbInputSpectra[i].counts / smbInputSpectra[i].exposure, 0.0),
        line=:steppost, label="SMB input")
    if (size(smbCentroidedOutputSpectra[i].locations)[1] > 0)
        sticks!(smbCentroidedOutputSpectra[i].locations,
            smbCentroidedOutputSpectra[i].counts / smbInputSpectra[i].exposure,
            label="SMB centroided output", m=2)
    end
    plot!(xlabel = "m/z (Th)", ylabel = "ion count density",
        xlims=(602, 605),
        ylims=(-0.05 * maxIntensity, 1.05 * maxIntensity))
end

### Compare mzML input to 'seamass-restore --centroid' mzMLb output

In [ ]:
mzmlCentroidedOutputSpectra = SeaMass.MzmlSpectrum[]
for i = 1:100
    push!(mzmlCentroidedOutputSpectra, SeaMass.MzmlSpectrum(
            "../data/out/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500" * 
                "/6.smb2mzmlb/P02U_Swath_1__mzWindow_602_605__scanTime_2300_3500.mzMLb", i))
end

@manipulate for i = slider(1:100, label="Spectrum", value=maxIntensitySpectrum, continuous_update=true)
    plot(mzmlInputSpectra[i].mzs, mzmlInputSpectra[i].intensities, label="mzML input", m=2)
    if (size(mzmlCentroidedOutputSpectra[i].mzs)[1] > 0)
        sticks!(mzmlCentroidedOutputSpectra[i].mzs, mzmlCentroidedOutputSpectra[i].intensities,
            label="mzML centroided output", m=2)
    end
    plot!(xlabel = "m/z (Th)", ylabel = "intensity",
        xlims=(602, 605),
        ylims=(-0.05 * maxIntensity, 1.05 * maxIntensity))
end